# Get data

In [3]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2014-01.parquet

--2023-07-18 10:58:12--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2014-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.155.128.6, 18.155.128.46, 18.155.128.187, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.155.128.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 171982098 (164M) [application/x-www-form-urlencoded]
Saving to: ‘yellow_tripdata_2014-01.parquet’

yellow_tripdata_201 100%[===================>] 164.01M  40.2MB/s    in 4.7s    

2023-07-18 10:58:18 (34.9 MB/s) - ‘yellow_tripdata_2014-01.parquet’ saved [171982098/171982098]



# Read Data

In [5]:
import pandas as pd

taxi = pd.read_parquet("yellow_tripdata_2014-01.parquet")
taxi.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2014-01-01 00:02:00,2014-01-01 00:04:00,6,0.0,1,None,146,146,1,3.5,0.5,0.5,0.02,0.0,0.0,4.52,NaN,None
1,2,2014-01-01 00:06:00,2014-01-01 00:09:00,5,0.0,1,None,146,146,1,3.5,0.5,0.5,0.05,0.0,0.0,4.55,NaN,None
2,2,2014-01-01 00:10:00,2014-01-01 00:13:00,5,0.0,1,None,146,146,1,3.5,0.5,0.5,0.08,0.0,0.0,4.58,NaN,None
3,2,2014-01-01 00:54:00,2014-01-01 00:55:00,5,0.0,1,None,264,264,2,2.5,0.5,0.5,0.00,0.0,0.0,3.50,NaN,None
4,1,2014-01-01 00:29:18,2014-01-01 00:35:13,2,1.8,1,N,229,262,2,7.5,0.5,0.5,0.00,0.0,0.0,8.50,NaN,None
